In [1]:
# import required libries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
# define required path
curated_data = '../../data/curated/'
raw_data = '../../data/raw/'

/home/shunranw/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# read csv files
liveable = pd.read_csv(curated_data + 'Liveability_rank.csv', index_col = 0)
affordable = pd.read_csv(curated_data + 'Affordability_rank.csv', index_col = 0)

In [3]:
# code from tut02
# sf stands for shape file
sf = gpd.read_file(raw_data + "suburbs_shape/vic_localities.shp")

In [4]:
# Convert the geometry shaape to to latitude and longitude
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [5]:
# (y, x) since we want (lat, long)
sf['centroid'] = sf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))

In [6]:
# find top 5 affordable suburbs
affordable_suburbs = list(affordable['Suburb'].head(5))

In [7]:
# find top 5 liveable suburbs
liveable_suburbs = list(liveable['Suburb'].head(5))

In [8]:
# prepare for merge
sf['LOC_NAME'] = sf['LOC_NAME'].str.upper()

In [9]:
# select the top 5 affordable suburbs
top5_aff = sf.query('LOC_NAME in @affordable_suburbs')

In [10]:
# select the top 5 liveable suburbs
top5_live = sf.query('LOC_NAME in @liveable_suburbs')

In [11]:
map_aff = folium.Map(location=[-37.02, 146.76], tiles="Stamen Terrain", zoom_start=10)

In [12]:
# draw the 5 subrubs in the map
for zone_name, coord in top5_aff[['LOC_NAME', 'centroid']].values:
    map_aff.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
map_aff.save('../../plots/affordable_Map.html')

In [13]:
map_live = folium.Map(location=[-37.02, 146.76], tiles="Stamen Terrain", zoom_start=10)

In [14]:
# draw the 5 subrubs in the map
for zone_name, coord in top5_live[['LOC_NAME', 'centroid']].values:
    map_live.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
map_live.save('../../plots/liveable_Map.html')